# lora-midm-7b-nsmc-review-understanding

1. KT-AI/midm-bitext-S-7B-inst-v1 를 영화 리뷰 이해에 미세 튜닝

- nsmc-review-2000.json (학습)
- midm-eval-1000.json (테스트)


종속적인 필요 내용
- huggingface 계정 설정 및 llama-2 사용 승인
- 로깅을 위한 wandb

In [ ]:
pip install transformers peft accelerate optimum bitsandbytes trl wandb einops

In [ ]:
import os
from dataclasses import dataclass, field
from typing import Optional
import re

import torch
import tyro
from accelerate import Accelerator
from datasets import load_dataset, Dataset
from peft import AutoPeftModelForCausalLM, LoraConfig
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)

from trl import SFTTrainer

from trl.trainer import ConstantLengthDataset

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset

dataset_name = "nsmc"
dataset = load_dataset(dataset_name)

nsmc_train=dataset['train'][:2000]

In [ ]:
import json

train_list = [{'input': doc, 'output': label} for doc, label in zip(nsmc_train['document'], nsmc_train['label'])]

for item in train_list:
    item['output'] = '부정' if item['output'] == 0 else '긍정'

train = json.dumps(train_list, ensure_ascii=False)
print(train)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
file_path = '/content/drive/MyDrive/nlp/nsmc-review-2000.json'

with open(file_path, 'w', encoding='utf-8') as file:
    file.write(train)

# 매개 변수 설정

In [ ]:
@dataclass
class ScriptArguments:
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "the cache dir"}
    )
    model_name: Optional[str] = field(
        default="KT-AI/midm-bitext-S-7B-inst-v1", metadata={"help": "the model name"}
    )

    dataset_name: Optional[str] = field(
        default=None,
        metadata={"help": "the dataset name"},
    )
    seq_length: Optional[int] = field(
        default=1024, metadata={"help": "the sequence length"}
    )
    num_workers: Optional[int] = field(
        default=8, metadata={"help": "the number of workers"}
    )
    training_args: TrainingArguments = field(
        default_factory=lambda: TrainingArguments(
            output_dir="./results",
            # max_steps=500,
            logging_steps=20,
            # save_steps=10,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=2,
            gradient_checkpointing=False,
            group_by_length=False,
            learning_rate=1e-4,
            lr_scheduler_type="cosine",
            # warmup_steps=100,
            warmup_ratio=0.03,
            max_grad_norm=0.3,
            weight_decay=0.05,
            save_total_limit=20,
            save_strategy="epoch",
            num_train_epochs=1,
            optim="paged_adamw_32bit",
            fp16=True,
            remove_unused_columns=False,
            report_to="wandb",
        )
    )

    packing: Optional[bool] = field(
        default=True, metadata={"help": "whether to use packing for SFTTrainer"}
    )

    peft_config: LoraConfig = field(
        default_factory=lambda: LoraConfig(
            r=8,
            lora_alpha=16,
            lora_dropout=0.05,
            target_modules=["c_attn", "c_proj", "c_fc"],
            bias="none",
            task_type="CAUSAL_LM",
        )
    )

    merge_with_final_checkpoint: Optional[bool] = field(
        default=False, metadata={"help": "Do only merge with final checkpoint"}
    )

# 유틸리티

In [ ]:
def chars_token_ratio(dataset, tokenizer, nb_examples=400):
    """
    Estimate the average number of characters per token in the dataset.
    """
    total_characters, total_tokens = 0, 0
    for _, example in tqdm(zip(range(nb_examples), iter(dataset)), total=nb_examples):
        text = prepare_sample_text(example)
        total_characters += len(text)
        if tokenizer.is_fast:
            total_tokens += len(tokenizer(text).tokens())
        else:
            total_tokens += len(tokenizer.tokenize(text))

    return total_characters / total_tokens


def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# 데이터 로딩

In [ ]:
def prepare_sample_text(example):
    """Prepare the text from a sample of the dataset."""

    prompt_template = """###System;{System}
    ###User;{User}
    ###Midm;{Midm}"""

    default_system_msg = (
        "너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다."
    )

    text = (
        prompt_template.format(System=default_system_msg, User=example["input"],Midm=example["output"])
    )

    return text

In [ ]:
def create_datasets(tokenizer, args):
    train_data = Dataset.from_json(args.dataset_name)

    chars_per_token = chars_token_ratio(train_data, tokenizer)
    print(f"The character to token ratio of the dataset is: {chars_per_token:.2f}")

    train_dataset = ConstantLengthDataset(
        tokenizer,
        train_data,
        formatting_func=prepare_sample_text,
        infinite=True,
        seq_length=args.seq_length,
        chars_per_token=chars_per_token,
    )
    return train_dataset

# 미세 튜닝용 모델 로딩

In [ ]:
script_args = ScriptArguments(
    num_workers=2,
    seq_length=200,
    dataset_name='/content/drive/MyDrive/nlp/nsmc-review-2000.json',
    model_name='KT-AI/midm-bitext-S-7B-inst-v1'
    )

In [ ]:
script_args.training_args.logging_steps = 50
script_args.training_args.max_steps = 200
script_args.training_args.output_dir = '/content/drive/MyDrive/nlp/lora-midm-7b-nsmc-review-understanding'
script_args.training_args.run_name = 'midm-7b-nsmc-review-understanding'

In [ ]:
print(script_args)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

원본인 'KT-AI/midm-bitext-S-7B-inst-v1' 는 *.bin 형태로 모델을 제공한다.
- 코랩에서 CPU 메모리 부족 발생

해결책
- safetensors로 변환한 모델을 업로드 하고 이를 사용하기로 한다.

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    script_args.model_name,
    quantization_config=bnb_config,
    device_map="auto",  # {"": Accelerator().local_process_index},
    trust_remote_code=True,
    use_auth_token=True,
    cache_dir=script_args.cache_dir, #구글 드라이브 경로로 다운받지 않는게 좋음(캐시로 동작 X)
)
base_model.config.use_cache = False

In [ ]:
base_model

In [ ]:
peft_config = script_args.peft_config

In [ ]:
peft_config

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    'KT-AI/midm-bitext-S-7B-inst-v1',
    # script_args.model_name,
    trust_remote_code=True,
    cache_dir=script_args.cache_dir,
)

if getattr(tokenizer, "pad_token", None) is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  # Fix weird overflow issue with fp16 training

tokenizer.add_special_tokens(dict(bos_token='<s>'))

base_model.config.pad_token_id = tokenizer.pad_token_id
base_model.config.bos_token_id = tokenizer.bos_token_id

In [ ]:
training_args = script_args.training_args

In [ ]:
train_dataset = create_datasets(tokenizer, script_args)

In [ ]:
len(train_dataset)

In [ ]:
trainer = SFTTrainer(
    model=base_model,
    train_dataset=train_dataset,
    eval_dataset=None,
    peft_config=peft_config,
    packing=script_args.packing,
    max_seq_length=script_args.seq_length,
    tokenizer=tokenizer,
    args=training_args,
)

In [ ]:
base_model.device

In [ ]:
print_trainable_parameters(base_model)

midm 모델을 주문 문장 이해에 적용시 특징
- 모델 로딩 과정에서 CPU도 5.1기가, 디스크 42.4기가, GPU 메모리: 7,4 기가

구글 코랩 T-4 GPU: 300스텝 (13:47초 예상)

시퀀스 길이 384의 경우
- 14.7 G / 15.0 G 사용
- 메모리 오버플로우 발생시 이보다 줄일 것

In [ ]:
trainer.train()

In [ ]:
script_args.training_args.output_dir

In [ ]:
trainer.save_model(script_args.training_args.output_dir)

In [ ]:
# 모델 업로드
trainer.push_to_hub("yaeeun/lora-midm-7b-nsmc-review")

모델 ID: yaeeun/lora-midm-7b-nsmc-review-understanding

# 추론 테스트

In [ ]:
from transformers import pipeline, TextStreamer

In [ ]:
instruction_prompt_template = """###System;다음은 관람객들이 쓴 영화 리뷰다. 이를 분석하여 고객의 평가가 긍정, 부정 둘 중 어떤 쪽인지 추출하고자 한다.
분석 결과를 완성해주기 바란다.

### 리뷰: {0} ### 분석 결과:
"""

prompt_template = """###System;{System}
###User;{User}
###Midm;"""

default_system_msg = (
    "너는 사용자가 작성한 리뷰의 긍정 또는 부정을 판단해야 한다."
)

In [ ]:
evaluation_queries = [
    "지루하지는 않은데 완전 막장임... 돈주고 보기에는....",
    "3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??",
    "한국독립영화의 한계 그렇게 아버지가 된다와 비교됨",
    "이별의 아픔뒤에 찾아오는 새로운 인연의 기쁨 But, 모든 사람이 그렇지는 않네..",
    "괜찮네요오랜만포켓몬스터잼밌어요",
    "	꽤 재밌게 본 영화였다!",
]

In [ ]:
def wrapper_generate(model, input_prompt, do_stream=False):
    data = tokenizer(input_prompt, return_tensors="pt")
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)
    input_ids = data.input_ids[..., :-1]
    with torch.no_grad():
        pred = model.generate(
            input_ids=input_ids.cuda(),
            streamer=streamer if do_stream else None,
            use_cache=True,
            max_new_tokens=float('inf'),
            do_sample=False
        )
    decoded_text = tokenizer.batch_decode(pred, skip_special_tokens=True)
    decoded_text = decoded_text[0].replace("<[!newline]>", "\n")
    return (decoded_text[len(input_prompt):])

In [ ]:
eval_dic = {i:wrapper_generate(model=base_model, input_prompt=prompt_template.format(System=default_system_msg, User=evaluation_queries[i]))for i, query in enumerate(evaluation_queries)}

In [ ]:
print(eval_dic[0])